In [1]:
import os
import json
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from data_utils.dataset import LocalImageDataset, get_split_indices
from model.ResNetSR import ResNetSR
from model.utils import get_device, train_model_single_epoch, validate_model_single_epoch, save_checkpoint, save_samples, CombinedLoss

In [2]:
with open("data/test_indices.json", "r") as f:
    test_indices = json.load(f)

In [3]:
len(test_indices)

594

In [4]:
INPUT_DIR = "data/resolution_128"
TARGET_DIR = "data/resolution_512"
NUM_EPOCHS = 30
BATCH_SIZE = 64
LEARNING_RATE = 0.01

GRAD_CLIP = 1
EDGE_WEIGHT = 0.3
PSNR_WEIGHT = 0.3

history = {
    "train_loss": [],
    "train_psnr": [],
    "val_loss": [],
    "val_psnr": []
}

In [5]:
num_images = len([f for f in os.listdir(INPUT_DIR) if f.endswith(('.jpg', '.png', '.jpeg', '.webp'))])
train_indices, val_indices = get_split_indices(num_images, test_indices)

train_loader = DataLoader(LocalImageDataset(INPUT_DIR, TARGET_DIR, train_indices), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(LocalImageDataset(INPUT_DIR, TARGET_DIR, val_indices), batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(LocalImageDataset(INPUT_DIR, TARGET_DIR, test_indices), batch_size=BATCH_SIZE, shuffle=False)

In [6]:
best_val_loss = float('inf')

save_ckpt = 2
checkpoint_dir = "ckpt"

n_samples = 5
samples_to_visualize = DataLoader(LocalImageDataset(INPUT_DIR, TARGET_DIR, test_indices), batch_size=n_samples, shuffle=False)

sample_dir = "samples"

train_losses = []
val_losses = []

In [7]:
device = get_device()
model = ResNetSR(upscale_factor=4, num_res_blocks=2, num_channels=1, num_features=32)
model = model.to(device)
mae_loss = nn.L1Loss()
criterion = CombinedLoss(mae_loss, EDGE_WEIGHT)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=60)

In [ ]:
history = {
    "train_loss": [],
    "train_psnr": [],
    "val_loss": [],
    "val_psnr": []
}

for epoch in range(1, 2):
    train_loss, train_psnr = train_model_single_epoch(model, train_loader, criterion, optimizer, device, scaler=None, grad_clip=GRAD_CLIP)
    val_loss, val_psnr = validate_model_single_epoch(model, val_loader, criterion, device)
    current_lr = scheduler.get_last_lr()[0]
    print(f"[Epoch {epoch}/{NUM_EPOCHS}]",
            f"Train Loss: {train_loss:.4f}, PSNR: {train_psnr:.2f} | "
            f"Val Loss: {val_loss:.4f}, PSNR: {val_psnr:.2f} | "
            f"lr: {round(current_lr, 5)}")

    if epoch % save_ckpt == 0:
        save_checkpoint(epoch+1, model, optimizer, history, checkpoint_dir)

        save_samples(epoch+1, model, val_loader, device, sample_dir, samples_to_visualize)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        save_checkpoint(0, model, optimizer, history, checkpoint_dir)

    scheduler.step(val_loss)

    history["train_loss"].append(train_loss)
    history["train_psnr"].append(train_psnr)
    history["val_loss"].append(val_loss)
    history["val_psnr"].append(val_psnr)

Training:   4%|▍         | 3/67 [00:44<20:33, 19.28s/it, loss=10.6, psnr=-15.9]

In [ ]:
# def train_model_for_resolution_pairs(resol_1, resol_2, psnr_weight=PSNR_WEIGHT):
#     INPUT_DIR = f"data/resolution_{resol_1}"
#     TARGET_DIR = f"data/resolution_{resol_2}"

#     num_images = len([f for f in os.listdir(INPUT_DIR) if f.endswith(('.jpg', '.png', '.jpeg', '.webp'))])
#     train_indices, val_indices = get_split_indices(num_images, test_indices)

#     train_loader = DataLoader(LocalImageDataset(INPUT_DIR, TARGET_DIR, train_indices), batch_size=BATCH_SIZE, shuffle=True)
#     val_loader = DataLoader(LocalImageDataset(INPUT_DIR, TARGET_DIR, val_indices), batch_size=BATCH_SIZE, shuffle=False)
#     test_loader = DataLoader(LocalImageDataset(INPUT_DIR, TARGET_DIR, test_indices), batch_size=BATCH_SIZE, shuffle=False)

#     best_val_loss = float('inf')

#     save_ckpt = 2
#     checkpoint_dir = f"ckpt/resolution_{resol_1}_{resol_2}"
#     os.makedirs(checkpoint_dir, exist_ok=True)

#     n_samples = 5
#     samples_to_visualize = DataLoader(LocalImageDataset(INPUT_DIR, TARGET_DIR, test_indices), batch_size=n_samples, shuffle=False)

#     sample_dir = f"samples_{resol_1}_{resol_2}"


#     device = get_device()
#     model = ResNetSR(upscale_factor=2, num_res_blocks=2, num_channels=1, num_features=32)
#     model = model.to(device)
#     mae_loss = nn.L1Loss()
#     criterion = CombinedLoss(mae_loss, EDGE_WEIGHT, psnr_weight)
#     optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
#     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#         optimizer, 
#         mode='max',  # Focus on maximizing PSNR
#         factor=.2,  # More aggressive reduction
#         patience=2,   # Reduce LR sooner
#         threshold=0.01,  # Smaller threshold for improvement
#         verbose=True
#     )

#     for epoch in range(1, NUM_EPOCHS + 1):
#         train_loss, train_psnr = train_model_single_epoch(model, train_loader, criterion, optimizer, device, scaler=None, grad_clip=GRAD_CLIP)
#         val_loss, val_psnr = validate_model_single_epoch(model, val_loader, criterion, device)
#         current_lr = scheduler.get_last_lr()[0]
#         print(f"[Epoch {epoch}/{NUM_EPOCHS}]",
#                 f"Train Loss: {train_loss:.4f}, PSNR: {train_psnr:.2f} | "
#                 f"Val Loss: {val_loss:.4f}, PSNR: {val_psnr:.2f} | "
#                 f"lr: {round(current_lr, 5)}")

#         if epoch % save_ckpt == 0:
#             save_checkpoint(epoch+1, model, optimizer, history, checkpoint_dir)

#             save_samples(epoch+1, model, val_loader, device, sample_dir, samples_to_visualize)

#         if val_loss < best_val_loss:
#             best_val_loss = val_loss
#             save_checkpoint(0, model, optimizer, history, checkpoint_dir)

#         scheduler.step(val_loss)

#         history["train_loss"].append(train_loss)
#         history["train_psnr"].append(train_psnr)
#         history["val_loss"].append(val_loss)
#         history["val_psnr"].append(val_psnr)

#     sns.lineplot(
#         x=range(1, epoch + 1),
#         y=history["train_psnr"],
#         label="Train PSNR",
#         color="blue"
#     )
#     sns.lineplot(
#         x=range(1, epoch + 1),
#         y=history["val_psnr"],
#         label="Validation PSNR",
#         color="orange"
#     )
#     plt.title("PSNR over epochs")
#     plt.xlabel("Epoch")
#     plt.ylabel("PSNR")
#     plt.legend()
#     plt.show()

#     os.mkdir("plots", exist_ok=True)
#     plt.savefig(f"plots/psnr_plot_{resol_1}_{resol_2}.png")

#     return history

In [15]:
psnr_weights = [0.1, 0.3, 0.5, 0.7, 0.9]
results = {}

for psnr_weight in psnr_weights:
    print(f"Training with PSNR weight: {psnr_weight}")
    history = train_model_for_resolution_pairs(64, 128, psnr_weight=psnr_weight)
    results[psnr_weight] = history

    # Plotting the results
    plt.figure(figsize=(12, 6))
    sns.lineplot(
        x=range(1, NUM_EPOCHS + 1),
        y=history["train_psnr"],
        label=f"Train PSNR (weight={psnr_weight})",
        color="blue"
    )
    sns.lineplot(
        x=range(1, NUM_EPOCHS + 1),
        y=history["val_psnr"],
        label=f"Validation PSNR (weight={psnr_weight})",
        color="orange"
    )
    plt.title(f"PSNR over epochs (PSNR weight={psnr_weight})")
    plt.xlabel("Epoch")
    plt.ylabel("PSNR")
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 6))
#     sns.lineplot(
#         x=range(1, NUM_EPOCHS + 1),
#         y=history["train_loss"],
#         label=f"Train Loss (weight={psnr_weight})",
#         color="blue"
#     )
#     sns.lineplot(
#         x=range(1, NUM_EPOCHS + 1),
#         y=history["val_loss"],
#         label=f"Validation Loss (weight={psnr_weight})",
#         color="orange"
#     )
#     plt.title(f"Loss over epochs (PSNR weight={psnr_weight})")
#     plt.xlabel("Epoch")
#     plt.ylabel("Loss")
#     plt.legend()
#     plt.show()

# # Save the results
# with open("grid_search_results.json", "w") as f:
#     json.dump(results, f)

Training with PSNR weight: 0.1


Training: 100%|██████████| 67/67 [00:09<00:00,  7.40it/s, loss=-1.05, psnr=13.3]  


[Epoch 1/30] Train Loss: 4.2355, PSNR: 2.67 | Val Loss: -0.9931, PSNR: 13.01 | lr: 0.01
Model checkpoint saved at ckpt/resolution_64_128/ckpt_0


Training: 100%|██████████| 67/67 [00:08<00:00,  8.21it/s, loss=-1.08, psnr=13.4]


[Epoch 2/30] Train Loss: -1.0885, PSNR: 13.49 | Val Loss: -1.1226, PSNR: 13.71 | lr: 0.01
Model checkpoint saved at ckpt/resolution_64_128/ckpt_3
Saved 5 samples at epoch 3 to samples_64_128/epoch_3_samples.png
Model checkpoint saved at ckpt/resolution_64_128/ckpt_0


...

Training: 100%|██████████| 67/67 [00:07<00:00,  9.48it/s, loss=-1.22, psnr=14.3]


[Epoch 29/30] Train Loss: -1.2059, PSNR: 14.21 | Val Loss: -1.2073, PSNR: 14.22 | lr: 0.0


Training: 100%|██████████| 67/67 [00:06<00:00, 10.08it/s, loss=-1.19, psnr=14.1]


[Epoch 30/30] Train Loss: -1.2059, PSNR: 14.21 | Val Loss: -1.2073, PSNR: 14.22 | lr: 0.0
Model checkpoint saved at ckpt/resolution_64_128/ckpt_31
Saved 5 samples at epoch 31 to samples_64_128/epoch_31_samples.png


ValueError: All arrays must be of the same length

In [ ]:
train_model_for_resolution_pairs(128, 256)

Training: 100%|██████████| 67/67 [00:34<00:00,  1.95it/s, loss=0.226, psnr=15.9]


[Epoch 1/30] Train Loss: 9.4055, PSNR: 6.08 | Val Loss: 0.2283, PSNR: 15.72 | lr: 0.01
Model checkpoint saved at ckpt/ckpt_0


Training: 100%|██████████| 67/67 [00:27<00:00,  2.41it/s, loss=0.193, psnr=16.3]


[Epoch 2/30] Train Loss: 0.2054, PSNR: 16.15 | Val Loss: 0.1873, PSNR: 16.52 | lr: 0.01
Model checkpoint saved at ckpt/ckpt_3
Saved 5 samples at epoch 3 to samples_128_256/epoch_3_samples.png
Model checkpoint saved at ckpt/ckpt_0


Training: 100%|██████████| 67/67 [00:27<00:00,  2.42it/s, loss=0.152, psnr=17.1]


...

[Epoch 28/30] Train Loss: 0.0786, PSNR: 20.35 | Val Loss: 0.0785, PSNR: 20.35 | lr: 0.00016
Model checkpoint saved at ckpt/ckpt_29
Saved 5 samples at epoch 29 to samples_128_256/epoch_29_samples.png
Model checkpoint saved at ckpt/ckpt_0


Training: 100%|██████████| 67/67 [00:55<00:00,  1.21it/s, loss=0.0782, psnr=20.4]


[Epoch 29/30] Train Loss: 0.0785, PSNR: 20.36 | Val Loss: 0.0784, PSNR: 20.35 | lr: 0.00016
Model checkpoint saved at ckpt/ckpt_0


Training: 100%|██████████| 67/67 [00:27<00:00,  2.42it/s, loss=0.0795, psnr=20.3]


[Epoch 30/30] Train Loss: 0.0783, PSNR: 20.37 | Val Loss: 0.0782, PSNR: 20.37 | lr: 8e-05
Model checkpoint saved at ckpt/ckpt_31
Saved 5 samples at epoch 31 to samples_128_256/epoch_31_samples.png
Model checkpoint saved at ckpt/ckpt_0


In [ ]:
train_model_for_resolution_pairs(256, 512)

Training: 100%|██████████| 67/67 [2:36:16<00:00, 139.94s/it, loss=0.135, psnr=19.4]   


[Epoch 1/30] Train Loss: 19.0649, PSNR: -2.96 | Val Loss: 0.1273, PSNR: 20.24 | lr: 0.01
Model checkpoint saved at ckpt/ckpt_0


Training: 100%|██████████| 67/67 [41:02<00:00, 36.76s/it, loss=0.16, psnr=16.9]  


[Epoch 2/30] Train Loss: 0.2267, PSNR: 17.36 | Val Loss: 0.3321, PSNR: 10.29 | lr: 0.01
Model checkpoint saved at ckpt/ckpt_3
Saved 5 samples at epoch 3 to samples_256_512/epoch_3_samples.png


Training: 100%|██████████| 67/67 [1:00:27<00:00, 54.15s/it, loss=0.1, psnr=21.7]    


...

Training: 100%|██████████| 67/67 [1:18:47<00:00, 70.57s/it, loss=0.0949, psnr=21.7] 


[Epoch 6/30] Train Loss: 0.3516, PSNR: 18.81 | Val Loss: 0.0965, PSNR: 21.19 | lr: 0.01
Model checkpoint saved at ckpt/ckpt_7
Saved 5 samples at epoch 7 to samples_256_512/epoch_7_samples.png
Model checkpoint saved at ckpt/ckpt_0


Training: 100%|██████████| 67/67 [40:51<00:00, 36.58s/it, loss=0.0752, psnr=22]  


[Epoch 7/30] Train Loss: 0.0806, PSNR: 22.01 | Val Loss: 0.0748, PSNR: 22.15 | lr: 0.005
Model checkpoint saved at ckpt/ckpt_0


Training: 100%|██████████| 67/67 [1:46:44<00:00, 95.60s/it, loss=0.0529, psnr=23.4]   
